In [ ]:
import pandas as pd
import os
import re
import numpy as np
from numpy import log 
from math import sqrt 

#librerias para texto
import nltk #Natural Language Tool Kit
#importamos tokenizador de texto
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer

#stemmer mediate algoritmo de porter
from nltk.stem import PorterStemmer 

#stemmer de libreria snowball
from nltk import SnowballStemmer 

import string #funciones adicionales para cadenas

#para reemplazo de caracteres especiales latinos
import unidecode 
import unicodedata

#para visualizacion
import matplotlib.pyplot as plt
import seaborn as sns

#modulos de sklearn
from sklearn.metrics import mean_squared_error
import sklearn.preprocessing
import sklearn.decomposition 
import sklearn.manifold
%matplotlib inline 

In [ ]:
#Establecemos directorio
dir=os.chdir('C:/Users/rfern/Desktop/Modulo 9/Tarea')

In [ ]:
#cargamos dataframe con texto limpio en formato pickle
df_paginas=pd.read_pickle('df_paginas.pkl')

In [ ]:
df_paginas.head()

In [ ]:
#Cargamos csv con el lexico afinn
lexicon=pd.read_csv('lexico_afinn.en.es.csv', sep=',', encoding='latin1')
lexicon[lexicon['Palabra'].str.contains('felicidad.*')]

In [ ]:
lexicon.head(50)

In [ ]:
#creamos funcion que identifica el puntaje para una cadena
def sentimiento(cadena):
    
    #creamos una cadena unificada con las palabras
    lexis=' '.join(lexicon.Palabra)
    
    #creamos lista con los puntajes
    punct=list(lexicon.Puntuacion)
    
    #creamos listas para almacenar la palabra y el puntaje
    palabra=[]
    puntaje=[]
    pu=[]
    
    #separamos la cadena en cada una de sus palabras
    palabras=cadena.split()
    pfinal=np.nan
    
    #iniciamos ciclo de busqueda y recuperacion de puntaje
    for p in palabras:
        try:
            #buscamos cada palabra en el lexico
            t=re.search(p,lexis).group()      
            #almacenamos la palabra en la lista de palabras
            palabra.append(t)
            #recuperamos el puntaje de la tabla
            #ojo que de la traduccion en ingles hay terminos repetidos (por ejemplo, cheerful y glad tienen la misma traduccion)
            pu=lexicon[lexicon['Palabra']==t]['Puntuacion'].mean()
            #almacenamos el puntaje en la lista correspondiente
            puntaje.append(pu)
            
        except AttributeError:
            continue
            
    #calculamos la media ignorando eventuales nan
    pfinal=np.nanmean(puntaje)
    
    #si el resultado es nan
    if np.isnan(pfinal)==True:
        #pasamos a ceros
        pfinal=np.nan_to_num(pfinal)
    else:
        #sino, manetenemos puntaje
        pfinal
     
    #retornamos resultado
    return(pfinal)     

In [ ]:
####LEMATIZACIÓN

#Con la lematización pasa algo similar, cortar palabras puede hacernos perder su riqueza.
snowball = SnowballStemmer(language='spanish')
#instanciamos el lematizador de porter
#porter= PorterStemmer()


####NORMALIZACIÓN DE TEXTO

def limpieza(texto):
    #pasamos a minusculas
    texto_norm=texto.lower()
    
    #removemos espacios al final y al inicio de cada cadena
    texto_norm=texto_norm.strip()
    
    #removemos dobles espacios: ojo que si los eliminamos, concatenamos palabras
    texto_norm=re.sub('\s\s',' ', texto_norm)
    
    #nuevo_texto = unidecode.unidecode(nuevo_texto)
    texto_norm = unicodedata.normalize('NFD', texto_norm).encode('ascii', 'ignore').decode("utf-8")
    
    #retenemos caracteres alfanumericos: ojo que incluimos la ñ
    texto_norm=re.sub('[^A-Za-zñ]+', ' ', texto_norm)
    
     #resultado de la funcion
    return(texto_norm)

In [ ]:
df_paginas['cadena_limpia']=df_paginas['contenido_pagina_crudo'].apply(lambda x: limpieza(x))

In [ ]:
#implementamos funcion de asignación de polaridades
df_paginas['polaridad'] = df_paginas['cadena_limpia'].apply(lambda x: sentimiento(x))

In [ ]:
#revisamos cabecera
df_paginas.head(60)

In [ ]:
#revisamos comportamiento resumen de las polaridades
display(df_paginas.polaridad.describe())

In [ ]:
#Exploramos distribución de polaridades en historigrama
df_paginas.polaridad.hist(bins=10)

In [ ]:
def identity_tokenizer(text):
    return text

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer,
                      ngram_range=(2,3),
                      lowercase=False,
                      use_idf=True)

#ajustamos vectorizador a los datos
tfidf.fit(df_paginas.contenido_limpio)

#implementamos matriz tf-idf
tfidf_data = tfidf.transform(df_paginas.contenido_limpio)

#Transformamos el objeto en una data frame
tfidf_df =pd.DataFrame(tfidf_data.toarray(), columns=tfidf.get_feature_names())

tfidf_df.head(50)

In [ ]:
#Preparamos datos para modelamiento

#Variable dependiente
mean=df_paginas.polaridad.mean()
print(mean)
std=df_paginas.polaridad.std()
print(std)
label=(df_paginas[['polaridad']].astype(float)-mean)/std

#variables independientes
features=tfidf_df

In [ ]:
label.head()

In [ ]:
#segmentacion de muestras para train-test
features_train, features_test, label_train, label_test = sklearn.model_selection.train_test_split( 
    features,
    label,
    test_size=0.3, 
    random_state = 11
)

In [ ]:
#instanciamos y ajustamos regresion lineal
lr = sklearn.linear_model.LinearRegression()
lr.fit(features_train, label_train)

In [ ]:
#generamos predicciones
lr_predictions = lr.predict(features_test)

In [ ]:
#definimos funcion para calcular el mean absolute error
def mae(true, predicted):
    error = true-predicted
    absolute_error = np.abs(error)
    return absolute_error.mean()

In [ ]:
#calculamos el mae
mae(label_test,lr_predictions)

In [ ]:
#Definimos funcion para el root mean squared error (rmse)
def rmse(true, predicted):
    error = true-predicted
    squared_error = error**2
    return (squared_error.mean())**0.5

In [ ]:
#obtenemos el rmse
rmse(label_test,lr_predictions)

In [ ]:
def regression_report(y_true, y_pred):
    return {
        'mae': sklearn.metrics.mean_absolute_error(y_true, y_pred),
        'rmse': sklearn.metrics.mean_squared_error(y_true, y_pred)
    }

In [ ]:
lr_regression_report = regression_report(label_test, lr_predictions)
lr_regression_report

In [ ]:
#obtenemos coeficientes de regresion
coefs_score=pd.DataFrame(list(lr.coef_)).T

coefs_score=coefs_score.rename(columns={coefs_score.columns[0]: 'Beta'})
var_tab=pd.DataFrame(features.columns)
var_tab=var_tab.rename(columns={var_tab.columns[0]: 'Var'})
coef_tab=pd.concat([var_tab, coefs_score], axis=1)
coef_tab=coef_tab.sort_values(by=['Beta'], ascending=False).reset_index()
coef_tab.head(20)

In [ ]:
#los 20 ngramas positivos más importantes
print(coef_tab.head(20))

In [ ]:
#Los 20 ngramas negativos más importantes
print(coef_tab.tail(40))

In [ ]:
# MODELO 2:SUPPORT VECTOR MACHINES
#instanciamos y ajustamos
svm = sklearn.svm.SVR(kernel='linear') 
svm.fit(features_train, label_train)

In [ ]:
#obtenemos predicciones
svm_predictions = svm.predict(features_test)

In [ ]:
#generamos reporte
svm_regression_report = regression_report(label_test, svm_predictions)
svm_regression_report

In [ ]:
#obtenemos coeficientes de regresion
coefs_score=pd.DataFrame(list(svm.coef_)).T

coefs_score=coefs_score.rename(columns={coefs_score.columns[0]: 'Beta'})
var_tab=pd.DataFrame(features.columns)
var_tab=var_tab.rename(columns={var_tab.columns[0]: 'Var'})
coef_tab=pd.concat([var_tab, coefs_score], axis=1)
coef_tab=coef_tab.sort_values(by=['Beta'], ascending=False).reset_index()
coef_tab

In [ ]:
#Los 20 ngramas positivos más importantes
print(coef_tab.head(30))

In [ ]:
#Los 20 ngramas negativos más importantes
print(coef_tab.tail(20))